In [1]:
from __future__ import print_function
import ipywidgets as widgets 
import matplotlib.pyplot as plt
import numpy as np
import csv
import sympy
from numpy import array

### Mediator Parameters

$n$ : the number of times a jobs is executed for verification.

$PR \in [0,1] \subset \mathbb{R}$ : the penalty rate, used to determine the minimum D of both Job Creators and Resource Providers

$NDPR \in [0,1] \subset \mathbb{R}$ : the non-deterministic penalty rate, used to determine the Job Creators non-deterministic D

$baseFee = JCmediatorPrice + RPmediatorPrice$ : The fee the Mediator requires to be available for a mediation. A mediator can be used if `available < JCmediatorPrice + RPmediatorPrice`

$mediationFee = D-pi$ : The amount Mediator receives for mediating a given dispute. 

$ND\_Fee = NDPR \times pi-pi$ : The amount Mediator receives for mediating a given dispute and finding JC guilty of non-determinism. 

### Mediator Model

In [2]:
# Mediator Parameters
wC_r = widgets.IntSlider(min=0,max=100,step=1,value=1, description='C_r', continuous_update=False)
wn = widgets.IntSlider(min=0,max=10,step=1,value=1, description='n',continuous_update=False)
wPR = widgets.FloatSlider(min=0,max=100,step=.5,value=1, description='PR', continuous_update=False) #Penalty Rate
wPR = widgets.FloatSlider(min=0,max=100,step=.5,value=1, description='PR', continuous_update=False) #Penalty Rate
wNDPR = widgets.FloatSlider(min=0,max=100,step=.5,value=1, description='NDPR',continuous_update=False)#Non-deterministic Penalty rate
wp_ETarget = widgets.FloatSlider(min=0,max=1,step=.01,value=1, description='p_E',continuous_update=False)#p_E target slider

def MNDU(JCmediatorPrice, RPmediatorPrice):
    baseFee = JCmediatorPrice + RPmediatorPrice
    mediationFee = p_v*(p2+(1-p_E))*(p_a**n)*(D-pi)
    ND_Fee = p_v*(p2+(1-p_E))*(1-(p_a**n))*(NDD-pi) 
    return available + mediationFee + ND_Fee




### Solver Parameters

$matchFee$ = The amount a solver receives for finding a given match. 

### Solver Model

In [3]:
#Solver
def SNDU(): #solver nondeterminisitc utility
    matchFee = RPMatchingFee + JCMatchingFee
    return matchFee

# JC Model

### Job Creator parameters

$\pi = instructionCount*JO.rate + bandwidthUsage*JO.rate$ : The max pi the JC is willing to pay to have a job executed.

$p_{v}$ : probability the JC verifies result a1. 

$p_{va2}$ : probability the JC verifies result a2. This probability is 0 because it can just go straight to mediation. 

$C_v$ : cost to JC to verify result.

$B$ : the value or B JC gets when a job is performed correctly.

$matchPrice$ : the amount JC pays to Solver for finding a suitable RP. 

$mediatorPrice$ : The amount JC pays to mediator for being available. 

$p_a$ : probability that job returns answer in class $a1$

$p_2 = 1-p_a$ : probability that job returns answer in class $a2$

$D = \pi \cdot (PR + n)$ : the amount the Job Creator must post as a security D in order to get matched. It multiplied by the $n$, which as a reminder is the number of times the mediator will replicate a job. 

This model takes into account when the JC is attempting to gain utility though tricking the mediator into believing a RP did a job incorrectly and giving it compensation. The JC can trick the mediator by providing a job with a non-deterministic output. The output is in one of two answer classes, $a1$ or $a2$ with $P(a1)>>P(a2)$. If RP responds with $a1$ the JC will pay RP. If RP responds with $a2$ JC will request mediation, expecting the mediator to find the answer to be $a1$ and determine that the $RP$ must have cheated. 
We only need to considert two answer classes because when the Mediator is performing verification it assigns fault to the RP only if it finds there to be one answer which does not match the RP answer, if it finds two answers that differ from one another, regardless of if they match the RP answer, it faults the JC. This means that JC wants to maximize the chance that the Mediator will get the same answer reapeatedly which is not Bed by having additional answer classes.  

We need to make sure that the platform compensation model does not make this strategy feasible. 

**Assumptions**

* The job has probability p_a of resulting in an answer in class a1 and 1-p_a of resulting in class a2

In [4]:
# JC parameters
wpi = widgets.IntSlider(min=0,max=1000,step=1,value=1, continuous_update=False)
wp_v = widgets.FloatSlider(min=0,max=1,step=.1,value=1, description='p_v', continuous_update=False) # how often JC verifies
wC_v = widgets.FloatSlider(min=0,max=100,step=.1,value=0, description='C_v',continuous_update=False) # cost for JC to verify
wB = widgets.IntSlider(min=0,max=1000,step=1,value=2, continuous_update=False)
matchPrice = 1
mediatorPrice = 1

def JCU(n,PR,NDPR,C_r,  p_a,B,pi,p_v=1,C_v=0,  p_E=1):
    p2 = 1-p_a
    
    NDD = pi * NDPR + (pi* n)
     
    JC_HRP  =  p_v*p_E*p_a*(B-pi-C_v) # RP correctly (p_E) returns answer 1 (p_a) and pay up to max_pi (-pi).
    
    NDJC_HRP_Gain = p_v*p_E*p2*(p_a**n)*(B + pi -C_v) # receives gas pi because it had to spend some to request mediation.
    NDJC_HRP_Lose = p_v*p_E*p2*(1-(p_a**n))*(B-NDD-C_r-C_v) #RP returns answer 2 correctly or responds incorrectly (p2+(1-p_E)) and send to mediator who finds out nonDeterminism (1-p_a^n), pay fine and reward (f2+r)
    ND = NDJC_HRP_Gain + NDJC_HRP_Lose
    
    DRP_Gain = p_v*(1-p_E)*(p_a**n)*(pi - C_v) 
    DRP_Lose = p_v*(1-p_E)*(1-p_a**n)*(-NDD-C_r-C_v)# compute incorrectly (1-p_E) get sent to mediation, finds nondeterminism (1-p_a^n) get compensation.
#     DRP_scared = p_v*(1-p_E)*(-pi-C_v) # JC doesn't want to risk getting caught, just pay RP
#     DRP =  max(DRP_Gain+DRP_Lose, DRP_scared)
    DRP = DRP_Gain + DRP_Lose
    
    
    noID_HRP = (1-p_v)*p_E*(B-pi)
    noID_DRP = (1-p_v)*(1-p_E)*(-pi)
    noVerifyCost = noID_HRP + noID_DRP
    
    JCnonDet = JC_HRP + ND + DRP + noVerifyCost
    return JCnonDet


In [5]:
sn, sPR, sNDPR, sp_a, sB, spi, sp_v, sC_v, sp_E, sC_r= sympy.symbols('n PR NDPR p_a B pi p_v C_v p_E C_r', real=True)

JU = JCU(sn,sPR,sNDPR,sC_r,   sp_a,sB,spi,sp_v,sC_v,   sp_E)
display(JU)

JU1 = sympy.expand(JU)
display(JU1)

JU1 = sympy.simplify(JU)
display(JU1)


p_E*p_a*p_v*(B - C_v - pi) + p_E*p_a**n*p_v*(1 - p_a)*(B - C_v + pi) + p_E*p_v*(1 - p_a)*(1 - p_a**n)*(B - C_r - C_v - NDPR*pi - n*pi) + p_E*(1 - p_v)*(B - pi) + p_a**n*p_v*(1 - p_E)*(-C_v + pi) + p_v*(1 - p_E)*(1 - p_a**n)*(-C_r - C_v - NDPR*pi - n*pi) - pi*(1 - p_E)*(1 - p_v)

B*p_E - C_r*p_E*p_a*p_a**n*p_v + C_r*p_E*p_a*p_v + C_r*p_a**n*p_v - C_r*p_v - C_v*p_v - NDPR*p_E*p_a*p_a**n*p_v*pi + NDPR*p_E*p_a*p_v*pi + NDPR*p_a**n*p_v*pi - NDPR*p_v*pi - n*p_E*p_a*p_a**n*p_v*pi + n*p_E*p_a*p_v*pi + n*p_a**n*p_v*pi - n*p_v*pi - p_E*p_a*p_a**n*p_v*pi - p_E*p_a*p_v*pi + p_a**n*p_v*pi + p_v*pi - pi

-p_E*p_a*p_v*(-B + C_v + pi) - p_E*p_a**n*p_v*(p_a - 1)*(B - C_v + pi) - p_E*p_v*(p_a - 1)*(p_a**n - 1)*(-B + C_r + C_v + NDPR*pi + n*pi) - p_E*(B - pi)*(p_v - 1) + p_a**n*p_v*(C_v - pi)*(p_E - 1) - p_v*(p_E - 1)*(p_a**n - 1)*(C_r + C_v + NDPR*pi + n*pi) - pi*(p_E - 1)*(p_v - 1)

# Job Creator Strategy

### Upper Bounds on Job Creator Cheating

\begin{equation}
    \begin{alignedat}{3}
        % HJC_HRP
        \textbf{JCU} = & p_v\cdot p_E\cdot p_a \quad &&  && \cdot (B-\pi-C_{v}) \\
        % ND
        %  NDJC_HRP Gain  
        +& p_v\cdot p_E\cdot (1-p_a)  && \cdot (p_a^{n})  && \cdot (B + \pi -C_{v}) \\
        %  NDJC_HRP Loss  
        +& p_v\cdot p_E\cdot (1-p_a)  && \cdot (1-p_a^{n})  && \cdot (B-D-C_r-C_{v}) \\
        % DRP
        % Gain
        +& p_v\cdot (1-p_E) && \cdot (p_a^{n})  && \cdot (\pi - C_{v}) \\
        % Loss
        +& p_v\cdot (1-p_E) && \cdot (1-p_a^{n}) && \cdot (-D-C_r-C_{v}) \\
        % No verify
        +& (1-p_v)\cdot p_E &&  && \cdot (B-\pi) \\
        +& (1-p_v)\cdot (1-p_E)  &&  && \cdot (-\pi) \\
    \end{alignedat}
    \label{eq:JCU}
\end{equation}

Assume worst case scenario

$p_v = 1$ ; $p_E = 1$ ; $C_v = 0$ ; $C_r = 0$

In [6]:
sJU = JU.subs({sp_v:1, sp_E:1, sC_v:0, sC_r:0 })
display(sJU)
sJU = sympy.expand(sJU)
# display(JU)
sJU = sympy.simplify(sJU)
# display(JU)
sJU = sympy.collect(sJU,sp_a)
sJU = sympy.collect(sJU,sp_a**(sn))
sJU = sympy.collect(sJU,sp_a**(sn+1))
sJU = sympy.collect(sJU,spi)
display(sJU)


p_a*(B - pi) + p_a**n*(1 - p_a)*(B + pi) + (1 - p_a)*(1 - p_a**n)*(B - NDPR*pi - n*pi)

B + pi*(-NDPR - n + p_a*(NDPR + n - 1) + p_a**n*(NDPR + n + 1) + p_a**(n + 1)*(-NDPR - n - 1))

In [7]:
# set B = pi
display(sJU)
B0 = sJU.subs({sB:spi})
display(B0)
B0 = sympy.expand(B0)
display(B0)
B0 = sympy.simplify(B0)
display(B0)

B0 = sympy.collect(B0,sp_a)
B0 = sympy.collect(B0,sp_a**(sn))
B0 = sympy.collect(B0,sp_a**(sn+1))
display(B0)
B0 = sympy.simplify(B0)
display(B0)

B + pi*(-NDPR - n + p_a*(NDPR + n - 1) + p_a**n*(NDPR + n + 1) + p_a**(n + 1)*(-NDPR - n - 1))

pi*(-NDPR - n + p_a*(NDPR + n - 1) + p_a**n*(NDPR + n + 1) + p_a**(n + 1)*(-NDPR - n - 1)) + pi

-NDPR*p_a*p_a**n*pi + NDPR*p_a*pi + NDPR*p_a**n*pi - NDPR*pi - n*p_a*p_a**n*pi + n*p_a*pi + n*p_a**n*pi - n*pi - p_a*p_a**n*pi - p_a*pi + p_a**n*pi + pi

pi*(NDPR*p_a + NDPR*p_a**n - NDPR*p_a**(n + 1) - NDPR + n*p_a + n*p_a**n - n*p_a**(n + 1) - n - p_a + p_a**n - p_a**(n + 1) + 1)

pi*(-NDPR - n + p_a*(NDPR + n - 1) + p_a**n*(NDPR + n + 1) + p_a**(n + 1)*(-NDPR - n - 1) + 1)

-pi*(NDPR + n - p_a*(NDPR + n - 1) - p_a**n*(NDPR + n + 1) + p_a**(n + 1)*(NDPR + n + 1) - 1)

In [8]:
B1 = B0 - sp_a

B1 = sympy.expand(B1)
display(B1)
B1 = sympy.simplify(B1)
display(B1)
B1 = sympy.solve([B1], [sNDPR])
B1 = B1[sNDPR]
B1 = sympy.collect(B1, sp_a)
B1 = sympy.collect(B1, sp_a**sn)
B1 = sympy.collect(B1, sp_a**(sn+1))
display(B1)


-NDPR*p_a*p_a**n*pi + NDPR*p_a*pi + NDPR*p_a**n*pi - NDPR*pi - n*p_a*p_a**n*pi + n*p_a*pi + n*p_a**n*pi - n*pi - p_a*p_a**n*pi - p_a*pi - p_a + p_a**n*pi + pi

NDPR*p_a*pi + NDPR*p_a**n*pi - NDPR*p_a**(n + 1)*pi - NDPR*pi + n*p_a*pi + n*p_a**n*pi - n*p_a**(n + 1)*pi - n*pi - p_a*pi - p_a + p_a**n*pi - p_a**(n + 1)*pi + pi

(-n*pi + p_a*(n*pi - pi - 1) + p_a**n*(n*pi + pi) + p_a**(n + 1)*(-n*pi - pi) + pi)/(pi*(-p_a - p_a**n + p_a**(n + 1) + 1))

In [9]:
dB0 = sympy.diff(B0,sp_a)
display(dB0)

print("solve for PR")
sol_p_a1 = sympy.solve([dB0], [sNDPR])
display(sol_p_a1[sNDPR])
sol_p_a1 = sympy.collect(sol_p_a1[sNDPR], sp_a)
sol_p_a1 = sympy.collect(sol_p_a1, sp_a**sn)
sol_p_a1 = sympy.collect(sol_p_a1, sp_a**(sn+1))
display(sol_p_a1)


print("solve for p_a")
# sol_p_a1 = sympy.solve([dB0], [sp_a])
# display(sol_p_a1)

sol_p_a2 = sympy.solve([dB0], [sp_a**sn])
display(sol_p_a2[sp_a**sn])
sol_p_a2 = sympy.collect(sol_p_a2[sp_a**sn], sp_a)
display(sol_p_a2)

-pi*(-NDPR - n - n*p_a**n*(NDPR + n + 1)/p_a + 1 + p_a**(n + 1)*(n + 1)*(NDPR + n + 1)/p_a)

solve for PR


(n**2*p_a**n - n**2*p_a**(n + 1) + n*p_a + n*p_a**n - 2*n*p_a**(n + 1) - p_a - p_a**(n + 1))/(-n*p_a**n + n*p_a**(n + 1) - p_a + p_a**(n + 1))

(p_a*(n - 1) + p_a**n*(n**2 + n) + p_a**(n + 1)*(-n**2 - 2*n - 1))/(-n*p_a**n - p_a + p_a**(n + 1)*(n + 1))

solve for p_a


p_a*(NDPR + n - 1)/(NDPR*n*p_a - NDPR*n + NDPR*p_a + n**2*p_a - n**2 + 2*n*p_a - n + p_a)

p_a*(NDPR + n - 1)/(-NDPR*n - n**2 - n + p_a*(NDPR*n + NDPR + n**2 + 2*n + 1))

In [10]:
ddB0 = sympy.diff(dB0,sp_a)
display(ddB0)


ddB0 = sympy.simplify(ddB0)
display(ddB0)
ddB0 = sympy.collect(ddB0, sNDPR)
display(ddB0)
ddB0 = sympy.simplify(ddB0)
display(ddB0)

-pi*(-n**2*p_a**n*(NDPR + n + 1)/p_a**2 + n*p_a**n*(NDPR + n + 1)/p_a**2 + p_a**(n + 1)*(n + 1)**2*(NDPR + n + 1)/p_a**2 - p_a**(n + 1)*(n + 1)*(NDPR + n + 1)/p_a**2)

-n*p_a**(n - 2)*pi*(NDPR*n*p_a - NDPR*n + NDPR*p_a + NDPR + n**2*p_a - n**2 + 2*n*p_a + p_a + 1)

-n*p_a**(n - 2)*pi*(NDPR*(n*p_a - n + p_a + 1) + n**2*p_a - n**2 + 2*n*p_a + p_a + 1)

-n*p_a**(n - 2)*pi*(NDPR*(n*p_a - n + p_a + 1) + n**2*p_a - n**2 + 2*n*p_a + p_a + 1)

In [11]:
# Compare
n = 4
NDPR = 0
p = .5
b = 2
pi = 1
C_r = 0

ySJ = JCU(n=n,PR=0,NDPR=NDPR,C_r=C_r,   p_a=p,B=b,pi=pi,p_v=1,C_v=0,p_E=1)
print(ySJ)

ysJU = sJU.subs({sn:n, sNDPR:NDPR, sp_a:p, sB:b, spi:pi})
print(ysJU)

-0.34375
-0.343750000000000


In [41]:
dJU = sympy.diff(sJU,sp_a)
display(dJU)
dJU = sympy.simplify(dJU)
display(dJU)
# dJU = sympy.expand(dJU)
# display(dJU)
dJU = sympy.collect(dJU,(sNDPR+sn+1))
display(dJU)

pi*(NDPR + n + n*p_a**n*(NDPR + n + 1)/p_a - 1 + p_a**(n + 1)*(n + 1)*(-NDPR - n - 1)/p_a)

pi*(n*p_a**n*(NDPR + n + 1) + p_a*(NDPR + n - 1) - p_a**(n + 1)*(n + 1)*(NDPR + n + 1))/p_a

pi*(p_a*(NDPR + n - 1) + (n*p_a**n - p_a**(n + 1)*(n + 1))*(NDPR + n + 1))/p_a

In [60]:
sol_dJU_spa = sympy.solve([dJU], [sp_a**(sn+1)])

sol_dJU_spa = sol_dJU_spa[sp_a**(sn+1)]
display(sol_dJU_spa) 

# sol_PR = sympy.collect(sol_dJU_spa, sNDPR)
sol_PR = sympy.collect(sol_dJU_spa, sp_a**(sn))
sol_PR = sympy.collect(sol_PR, sp_a)
sol_PR = sympy.collect(sol_PR, sNDPR)
display(sol_PR)

(NDPR*n*p_a**n + NDPR*p_a + n**2*p_a**n + n*p_a + n*p_a**n - p_a)/(NDPR*n + NDPR + n**2 + 2*n + 1)

(p_a*(NDPR + n - 1) + p_a**n*(NDPR*n + n**2 + n))/(NDPR*(n + 1) + n**2 + 2*n + 1)

In [61]:
sol_dJU_NDPR = sympy.solve([dJU], [sNDPR])
tNDPR = sol_dJU_NDPR[sNDPR]
tNDPR = sympy.simplify(tNDPR)
display(tNDPR)


tNDPR = sympy.collect(tNDPR, sp_a)
tNDPR = sympy.collect(tNDPR, sp_a**sn)
tNDPR = sympy.collect(tNDPR, sp_a**(sn+1))

print("collect p_a terms")
display(tNDPR)



tNDPR2 = sol_dJU_NDPR[sNDPR]
tNDPR2 = sympy.collect(tNDPR2, sn)
# tNDPR2 = sympy.collect(tNDPR2, spn)
# tNDPR2 = sympy.collect(tNDPR2, sp_a**(sn+1))
print("collect n terms")
display(tNDPR2)

# sol_dJU_n = sympy.solve([dJU], [sn])
# display(sol_dJU_n)


(-n**2*p_a**n + n**2*p_a**(n + 1) - n*p_a - n*p_a**n + 2*n*p_a**(n + 1) + p_a + p_a**(n + 1))/(n*p_a**n - n*p_a**(n + 1) + p_a - p_a**(n + 1))

collect p_a terms


(p_a*(1 - n) + p_a**n*(-n**2 - n) + p_a**(n + 1)*(n**2 + 2*n + 1))/(n*p_a**n + p_a + p_a**(n + 1)*(-n - 1))

collect n terms


(n**2*(p_a**n - p_a**(n + 1)) + n*(p_a + p_a**n - 2*p_a**(n + 1)) - p_a - p_a**(n + 1))/(n*(-p_a**n + p_a**(n + 1)) - p_a + p_a**(n + 1))

In [ ]:
ddJU = sympy.diff(dJU,sp_a)
ddJU = sympy.simplify(ddJU)
display(ddJU)
ddJU = sympy.factor(ddJU)
display(ddJU)

In [ ]:
def plotdJU(n):
    
    plt.figure(2,figsize=(10,5))
    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    
    ydu = list(map(lambda p: float(sdJU.subs({sn:n, sp_a:p})), x))
    
    plt.plot(x,ydu, label="dU")
    
    plt.xlabel('p_a')
    plt.ylabel('dUtil') 
    
    plt.legend()
    plt.show
    

In [ ]:
# ui1 = widgets.HBox([wn,wPR])
# ui2 = widgets.HBox([wB,wpi])
# interactive_plot = widgets.interactive_output(plotdJU,{'n':wn});
# # output = interactive_plot.children[-1]
# # interactive_plot.layout.height = '400px'
# display(interactive_plot,ui1,ui2)

The Job Creators verification strategy is controlled by $p_v$, which is the probability of verifying a given result. 

The type of job, is captured by the $\pi$ and the $C_v$ terms. The price $\pi$ captures the complexity of execution, $C_v$ captures the complexity of verification.

A Resource Provider may choose to execute a job correctly or not. We capture this with the parameter $p_E$. If $p_E=1$ then the $RP$ executes all jobs correctly, if $p_E=0$ then it does not execute any jobs correctly.  

<!---While running non-deterministic jobs a Job Creator must decide whether the pay the $RP$ for a bad response, or risk getting caught in mediation. Currently the choice is made in the `max(RPfraud1,RPfraud2)` term. If the Job Creator is running deterministic jobs if it verifies and finds a bad result it will request mediation.--->

## Resource Providier

The Resource Provider only has one method to try to abuse the system. That is to return a result and hope that the Job Creator does not verify the job. If the Job Creator never verifies the Resource Provider will always cheat, and if it always verifies the Resource Provider will never cheat. However, the RP does not know how often the Job Creator will verify. 

### Resource Provider Model
The most RP can be paid for executing a job is $\pi = instructionCount*JO.rate + bandwidthUsage*JO.rate$. 

Jobs are only worth executing if the operating costs are covered by the Resource providers fees, which are at most $\pi$ for a given job. 

The Resource Provider does not only want to cover operating costs however, it additionally wants to make a profit. 

$profit = \pi-cost$

Since the price $\pi$ varies from job to job, the RP specifies a return on investment $roi$: 

$roi = \frac{\pi-cost}{\pi}$

For the model we need to estimate the cost of executing a job $cost$. We compute this with the given $\pi$ and required roi. This is reasonable since the JC specifies the $\pi$ and $\pi$ is ralated to job complexity, and the execution cost will increase with complexity.

$cost = \pi*(1-roi)$ 

The platform will only match a job to an RP if $\pi>requestedPrice$.

$requestedPrice$ covers the cost of execution. So in the analysis if $\pi<requestedPrice$ the point is infeasible. 

$requestedPrice$ is not used in the RP model, only computed. 

$requestedPrice = cost$

In a situation where the Job Creator is sending non-deterministic jobs, a Resource provider will lose 

$p_v \times (1-p_a) \times p_a^n\times D$

If the RP assumes $p_v=1$(JC always verifies), and that JC uses optimal $p_a$ given $n$ and $NDPR$ it can determine the maximum percent $I$ of its D that it will lose to the mediator being tricked. By adding $ I \times D$ to its $requestedPrice$ it can mitigate the damage done by cheating Job Creators. $I$ is an insurance percent against cheating. This results in a new $requestedPrice$ :  

$requestedPrice = \pi \times(1-roi) + D \times I $


When the RP is matched at its' $requestedPrice$ the $cost$ will effectively be reduced by $D \times I$ so the effective $cost$ used in the model is:

$cost = \pi*(1-roi) - D \times I $

In the `plot` function $requestedPrice$ is compared against $\pi$ to check if the point is valid. 


In [ ]:

wp_E = widgets.FloatSlider(min=0,max=1,step=.1,value=1,description='p_E', continuous_update=False) # probability that RP runs correctly
wroi = widgets.FloatSlider(min=0,max=1,step=.01,value=.5,description='roi', continuous_update=False) # what percent of reward is profit
winsurance = widgets.FloatSlider(min=0,max=1,step=.01,value=0 ,description='I',continuous_update=False) # % of deposit that can be recovered because execution is so cheap.
wC_f= widgets.FloatSlider(min=0,max=1000,step=.1,value=0, description='C_f',continuous_update=False) # cost for RP to be fraudulant 

matchPrice = 1
mediatorPrice = 1


def RPU(n,PR,NDPR,p_a,pi,p_v=1,C_f=0,p_E=1, ec=0, I=0):
    p2 = 1-p_a
#     D = pi * PR
#     D = pi * PR + pi*n
    D = pi * PR + (pi * n)
    
    
    execCost = ec
    
    insurance =  I

    #Cj is cost of execution for fraudulant job, picking a random number or "close" value
    fraudCost = C_f
    
    JC_HRP = p_v*p_E*p_a*(pi-execCost) #compute answer 1 (p_a) correctly (p_E) receive reward (pi)
    
    NDJC_HRP_Gain = p_v*p_E*p2*(1-p_a**n)*(pi-execCost) #compute answer 2 (p2) correctly (p_E) get sent to mediation, who finds nondeterminism (1-p_a^n) and recieve reward (r) and nonDet bonus (f2-f)*k kickback=k
    NDJC_HRP_Lose = p_v*p_E*p2*(p_a**n)*(-D-execCost) # compute answer 2 correctly (p2) get sent to mediation, who doesn't find nondeterminsm (p_a^n) and get fined (f)
    ND = NDJC_HRP_Gain + NDJC_HRP_Lose 
    
    DRP_Gain = p_v*(1-p_E)*(1-p_a**n)*(pi-fraudCost)# compute incorrectly (1-p_E) get sent to mediation, finds nondeterminism (1-p_a^n) get compensation.
    DRP_Lose = p_v*(1-p_E)*(p_a**n)*(-D-fraudCost) #compute incorrectly (1-p_E) get sent to mediation, who doesn't find nondeterminsm (p_a^n) and get fined (f)
    DRP = DRP_Gain+DRP_Lose
    
#     DRP_scared  = p_v*(1-p_E)*(pi-fraudCost) # JC doesn't want to risk getting caught, just pay RP. 
    
    #     noVerifypayout = (1-p_v)*pi #compute answer 1 (p_a) correctly (p_E) receive reward (pi)
    noID_HRP = (1-p_v) * p_E * (pi - execCost)
    noID_DRP = (1-p_v) * (1-p_E) * (pi - fraudCost)
    noID = noID_HRP + noID_DRP
    
    
    RCnonDet = JC_HRP + ND + DRP + insurance + noID
    return RCnonDet


In [ ]:
sn, sPR, sNDPR, sp_a, sB, spi, sp_v, sC_v, sp_E, sC_r= sympy.symbols('n PR NDPR p_a B pi p_v C_v p_E C_r', real=True)

rn,rPR,rNDPR,rp_a,rpi,rp_v,rC_f,rp_E, rec, rI = sympy.symbols('n PR NDPR p_a pi p_v C_f p_E ec I')

RU = RPU(rn,rPR,rNDPR,rp_a,rpi,rp_v,rC_f,rp_E, rec, rI)
display(RU)

In [ ]:
sRU = sympy.simplify(RU)
display(sRU)
sRU = sympy.collect(sRU,rp_a)
display(sRU)
sRU = sympy.collect(sRU,rp_a**(rn))
display(sRU)
sRU = sympy.collect(sRU,rp_a**(rn+1))
display(sRU)

In [ ]:
# Job Creator Strategy
display(RU)
RU0 = RU
RU1 = RU
sRUe0 = RU0.subs({rI:0, rp_E:0})
display(sRUe0)
sRUe1 = RU1.subs({rI:0, rp_E:1})
display(sRUe1)

sRUp = sRUe1 - sRUe0
display(sRUp)

sRUp = sympy.expand(sRUp)
display(sRUp)

sRUp = sympy.simplify(sRUp)
display(sRUp)

sRUp = sympy.collect(sRUp,rp_a)
display(sRUp)
sRUp = sympy.collect(sRUp,rp_a**(rn))
display(sRUp)
sRUp = sympy.collect(sRUp,rp_a**(rn+1))
display(sRUp)
sRUp = sympy.collect(sRUp,rpi)
display(sRUp)
sRUp = sympy.collect(sRUp,rp_v)
display(sRUp)


In [ ]:
sRU = RU.subs({rI:0})
display(sRU)

sRU = sympy.expand(sRU)
# display(JU)
sRU = sympy.simplify(sRU)
# display(JU)
sRU = sympy.collect(sRU,rp_a)
sRU = sympy.collect(sRU,rp_a**(rn))
sRU = sympy.collect(sRU,rp_a**(rn+1))
sRU = sympy.collect(sRU,rpi)
display(sRU)        

In [ ]:
dRU = sympy.diff(RU,rp_E)
display(dRU)

dRU = sympy.expand(dRU)
# display(dRU)

dRU = sympy.simplify(dRU)
# display(dRU)

dRU = sympy.collect(dRU,rp_a**(rn+1))
# display(dRU)

dRU = sympy.collect(dRU,rpi)
# display(dRU)

dRU = sympy.collect(dRU,rp_v)
display(dRU)

sol_dRU = sympy.solve([dRU], [rp_v])
display(sol_dRU)


In [ ]:
dRU_pa = sympy.diff(RU,rp_a)
display(dRU_pa)

dRU_pa = sympy.factor(dRU_pa)
display(dRU_pa)

sol_dRU_pa = sympy.solve([dRU_pa], [rp_a])
display(sol_dRU_pa)

In [ ]:
ddRU_pa = sympy.diff(dRU,rp_a)
display(ddRU_pa)

ddRU_pa = sympy.factor(ddRU_pa)
display(ddRU_pa)

In [ ]:
points = 51
x,step = np.linspace(0,1,points,retstep=True)
print("step size: %s" %step)

def plot(n,PR,NDPR,C_r,pi,b,p_v=1,C_v=0, p_E=1,roi=1,Insurance=0,C_f=0):   
    plt.figure(2,figsize=(20,5))
    
    plt.subplot(1,3,1)
    plt.grid(visible=True)
    
    yHJ = JCU(n=n,PR=PR,NDPR=NDPR,C_r=C_r, p_a=np.ones(points),B=b,pi=pi,p_v=1,C_v=C_v,p_E=p_E)
    yLJ = JCU(n=n,PR=PR,NDPR=NDPR,C_r=C_r, p_a=np.ones(points),B=b,pi=pi,p_v=p_v,C_v=C_v,p_E=p_E)
    yDJ = JCU(n,PR,NDPR,C_r=C_r, p_a=x,B=b,pi=pi,p_v=p_v,C_v=C_v, p_E=p_E)
    
    yPJ = JCU(n=n,PR=0,NDPR=NDPR,C_r=C_r,p_a=x,B=b,pi=pi,p_v=1,C_v=C_v,p_E=p_E)
    
    ySJ = JCU(n=n,PR=0,NDPR=NDPR,C_r=C_r,p_a=x,B=b,pi=pi,p_v=1,C_v=0,p_E=1)
    
    yVSJ = JCU(n=n,PR=0,NDPR=0,C_r=C_r,p_a=x,B=b,pi=pi,p_v=1,C_v=0,p_E=1)
    
    ySJ2 = JCU(n=n,PR=0,NDPR=NDPR,C_r=C_r,p_a=x,B=b,pi=pi,p_v=1,C_v=0,p_E=p_E)
    
#     ysJU = list(map(lambda p: float(sJU.subs({sn:n, sNDPR:0, sp_a:p, sB:b, spi:pi})),x))
#     ydJU  = list(map(lambda p: float(sdJU.subs({sn:n, sp_a:p})), x))
    
    
    D = pi * PR + (pi * n)
    I = D*Insurance
    
    ec = pi*(1-roi)
    
    
    yRU = RPU(n,PR,NDPR,p_a=x,pi=pi,p_v=p_v,C_f=C_f,p_E=p_E,ec=ec,I=I)
#     ydRU = list(map(lambda p: float(dRU.subs({rC_f:C_f, rec:ec, rp_a:p, rn:n, rp_v:p_v, rpi:pi, rPR:PR  })),x))
    
    ydRU_pa = list(map(lambda p: float(dRU_pa.subs({rp_a:p, rn:n, rp_v:p_v, rpi:pi, rPR:PR, rp_E:p_E  })),x))
    
    
    plt.plot(x,yHJ, label="HJ")
    plt.plot(x,yLJ, label="LJ")
    plt.plot(x,yDJ, label="DJ")
    plt.plot(x,yPJ, label="PJ")
#     plt.plot(x,ySJ, label="SJ")
#     plt.plot(x,yVSJ, label="VSJ")
#     plt.plot(x,ySJ2, label="SJ2")
    
    
#     plt.plot(x,ysJU, label="sJU")
#     plt.plot(x,ydJU, label="dJU")
   
    plt.plot(x,yRU, label="RU")
    
    plt.plot(x,ydRU_pa, label ="dRU_pa")    
    
    plt.xlabel('p_a')
    plt.ylabel('Util') 
    plt.legend()
    
    
    print("n: %s" %n)
    print("Price: %s" %pi)
    D = pi * PR + pi*n
    print("D: %s" %D)
    
    maxDJ = max(yDJ)
    print("maxDJ: %s" %maxDJ)
    
    maxRU = max(yRU)
    print("maxRU: %s" %maxRU)
    
    
    minRU = min(yRU)
    print("minRU: %s" %minRU)
    
    
    p_amaxJ = x[np.argmax(yDJ)]
    p2 = 1 - p_amaxJ
    print("p_a for JC max util: %s" %p_amaxJ)
    print("")
    
    p_aminRU = x[np.argmin(yRU)]
    print("p_a for RP min util: %s" %p_aminRU)
    print("")
    
    p_amaxRU = x[np.argmax(yRU)]
    print("p_a for RP max util: %s" %p_amaxRU)
    print("")
    
    
    print("Resource Provider")
    execCost = pi*(1-roi)
    requstedPrice = execCost + I 
    print("requstedPrice: %s" %requstedPrice)   
    print("pi > requstedPrice, Matchable? : %s" %(pi>requstedPrice))
    
    print("exec cost: %s" %ec)
    
    
    plt.subplot(1,3,2)
    plt.grid(visible=True)
    yRU_p_E = RPU(n,PR,NDPR,p_a=p_aminRU,pi=pi,p_v=p_v,C_f=C_f,p_E=x,ec=ec,I=I)
    plt.plot(x,yRU_p_E, label ="yRU wrt p_E")
    plt.xlabel('p_E')
    plt.ylabel('Util')
    plt.legend()
    
    plt.subplot(1,3,3)
    plt.grid(visible=True)
    yDJ_pv_mxJ = JCU(n,PR,NDPR,C_r=C_r, p_a=p_amaxJ,B=b,pi=pi,p_v=x,C_v=C_v, p_E=p_E)
    yDJ_pv_mnR = JCU(n,PR,NDPR,C_r=C_r, p_a=p_aminRU,B=b,pi=pi,p_v=x,C_v=C_v, p_E=p_E)
    yRU_pv_mnR_e0 = RPU(n,PR,NDPR,p_a=p_aminRU,pi=pi,p_v=x,C_f=C_f,p_E=0,ec=ec,I=I)
    yRU_pv_mnR_e1 = RPU(n,PR,NDPR,p_a=p_aminRU,pi=pi,p_v=x,C_f=C_f,p_E=1,ec=ec,I=I)
    
    plt.plot(x,yDJ_pv_mxJ, label ="yDJ wrt pv p_amxJ")
    plt.plot(x,yDJ_pv_mnR, label ="yDJ wrt pv p_amnR")
    plt.plot(x,yRU_pv_mnR_e0, label ="yRU wrt pv p_amxJ E0")
    plt.plot(x,yRU_pv_mnR_e1, label ="yRU wrt pv p_amnR E1")
    
    
    plt.xlabel('p_V')
    plt.ylabel('Util')
    
    plt.legend()
    plt.show
    
    display(dRU)
    print("min RU: ", dRU.subs({rC_f:C_f, rec:ec, rp_a:p_aminRU, rn:n, rp_v:p_v, rpi:pi, rPR:PR  }), "at p_a=%s" %p_aminRU) 

In [ ]:

# platform
ui1 = widgets.HBox([wn,wPR,wNDPR,wC_r]) 
# Job creator
ui21 = widgets.HBox([wB,wpi])
ui22 = widgets.HBox([wp_v,wC_v])
# Resource Provider
ui31 = widgets.HBox([wC_f,wp_E])
ui32 = widgets.HBox([wroi,winsurance])

# interactive_plot = widgets.interactive(plot, n=wn,PR=wPR,NDPR=wNDPR,pi=wpi,p_v=wp_v,cj=wcj, p_E=wp_E,roi=wroi,Insurance=winsurance);
interactive_plot = widgets.interactive_output(plot,{'n':wn,'PR':wPR,'NDPR':wNDPR,'C_r':wC_r,'pi':wpi,'b':wB,'p_v':wp_v,'C_v':wC_v, 'p_E':wp_E,'roi':wroi,'Insurance':winsurance,'C_f':wC_f });
# output = interactive_plot.children[-1]
# interactive_plot.layout.height = '900px'
display(interactive_plot,ui1,ui21,ui22,ui31,ui32)

# B Analysis

This section analyzes when the JC is sending real jobs that provide some benefit $B$ when they are successfully completed. The JC is attempting to increase its utility by adding a non-deterministic element to the jobs in an effort to have some jobs completed for free by accusing the RP of failing to execute them correctly. This could be done by changing the input so that some parts of the code are normally dorment and only activated when a certain data input is provided.

### Protocol Following model
If the JC follows the protocol its utility is the B, or value, it receives from the Job execution minus the amount it pays to the RP. This is assuming that the RP is playing by the rules of the game, otherwise the JC could have better utility since $\pi$ will be refunded and **B** will still be recieved since the mediator will return the result. 

This model is further simplified by the assumption that a Job can be checked for free, and that the JC can check the result of every job. This is a reasonable simplification since there are jobs whose verification cost is much less than the cost of execution, and we are interested in identifying the worst case scenario we need to protect against. 

### Non-Deterministic Model
The key element of this model is the amount of utility the JC can get for **free** from the system. 
The free utility is the probability that the RP got result 2 multiplied by the probability that the mediator got result 1 $n$ times multiplied by the job B, minus the probability that the RP got result 2 multiplied by the probability that the mediator did not get result 1 for any of $n$ executions multiplied by the D, whose value is determined by the pi set by the JC multiplied by the non-deterministic penalty rate set by the Mediator. 

This model is the best the JC can hope for. The RP could fail to complete jobs, in which case the JC loses the B, and either pays $pi$ or ND D.

### Evaluation

In [ ]:
def plot2(n,NDPR, pi,B,  p_v,C_v,p_E):
    points = 101
    x,step = np.linspace(0,1,points,retstep=True)
    
    
    yHJ = JCU(n=n,PR=0,NDPR=NDPR,C_r=0,p_a=np.ones(points),B=B,pi=pi,p_v=p_v,C_v=C_v,p_E=p_E)
    yDJ = JCU(n=n,PR=0,NDPR=NDPR,C_r=0,p_a=x,B=B,pi=pi,p_v=p_v,C_v=C_v,p_E=p_E)
    
    
    yHSJ = JCU(n=n,PR=0,NDPR=NDPR,C_r=0,p_a=np.ones(points),B=B,pi=pi,p_v=1,C_v=0,p_E=1)
    ySJ  = JCU(n=n,PR=0,NDPR=NDPR,C_r=0,p_a=x,B=B,pi=pi,p_v=1,C_v=0,p_E=1)
    ySJ0 = JCU(n=1,PR=0,NDPR=0,C_r=0,p_a=x,B=B,pi=pi,p_v=1,C_v=0,p_E=1)
    ySJ1 = JCU(n=2,PR=0,NDPR=0,C_r=0,p_a=x,B=B,pi=pi,p_v=1,C_v=0,p_E=1)
    ySJ2 = JCU(n=1,PR=0,NDPR=1,C_r=0,p_a=x,B=B,pi=pi,p_v=1,C_v=0,p_E=1)
    ySJ3 = JCU(n=2,PR=0,NDPR=1,C_r=0,p_a=x,B=B,pi=pi,p_v=1,C_v=0,p_E=1)
    
    
    plt.figure(3,figsize=(10,5))    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    
#     plt.plot(x,yPU, label="PU")
#     plt.plot(x,yNDU, label="NDU")
    
    plt.plot(x,yHJ, label="HJ : p_a=1")
    plt.plot(x,yDJ, label="DJ")
#     plt.plot(x,ySJ, label="SJ")
    
    plt.plot(x,yHSJ, label="HSJ :n=n,PR=PR")
    plt.plot(x,ySJ0, label="DJ :n=1,PR=0")
    plt.plot(x,ySJ1, label="DJ :n=2,PR=0")
    plt.plot(x,ySJ2, label="DJ :n=1,PR=1")
    plt.plot(x,ySJ3, label="DJ :n=2,PR=1")

    
    plt.xlabel('p_a')
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show
    
    print("Protocol util: %s" %max(yHJ))
    print("max ND util: %s" %max(yDJ))
    print("ND improvement over Protocol: %s" %(max(yDJ)-max(yHJ)))    
    print("%% improvement : %s %%" %(100*abs((max(yDJ)-max(yHJ))/max(yHJ))))
    print("optimal P(p_a): %s" %x[np.argmax(yDJ)])
    
    with open('JCND_file.csv', mode='w') as JCND_file:
        JCwriter = csv.writer(JCND_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        JCwriter.writerow(['p_a','JC Deterministic', 'JC Non-Deterministic'])
        
        for i in range(101):
            JCwriter.writerow([i/100-1/100, yHJ[i], yDJ[i] ])

    
    

In [ ]:
interactive_plot2 = widgets.interactive(plot2, n=wn,NDPR=wNDPR,   pi=wpi,B=wB,p_v=wp_v,C_v=wC_v,p_E=wp_E);
output = interactive_plot2.children[-1]
# output.layout.height = '420px'
interactive_plot2

#n = 4
#NDPR = 24? if pi is included in free job.

In [ ]:
x,step = np.linspace(0,1,51,retstep=True)

def plotRP(n,PR,NDPR,p_a,pi,b,p_v=1,C_v=0,roi=1,Insurance=0,C_f=0): 

    plt.figure(figsize=(10,5))
    
#     plt.subplot(1,1,1)
    plt.grid(visible=True)
    print("Comparison of RPU while varying p_E")
    
    print("step size: %s" %step)
    
    ec = pi*(1-roi)
    D = pi * PR + (pi * n)
    I = D*Insurance
    
    yRU = RPU(n,PR,NDPR,p_a,pi,p_v=p_v,C_f=C_f,p_E=x,ec=ec,I=I)
#     yRNDD = list(map(lambda p_E: RPU(n,PR,NDPR,p_a,pi,p_v=p_v,C_f=C_f,p_E=p_E,ec=ec,I=I),x))
    
    print("n,PR,NDPR,pi,p_a,p_v=p_v,p_E=p_E,ec=ec,I=Insurance,C_f=C_f")
    print( n,PR,NDPR,pi,p_a,p_v,   "p_E",   ec,   I,          C_f)
   
    plt.plot(x,yRU, label="RPU")
    plt.xlabel('p_E')
    plt.ylabel('Util') 
    
    
    maxRU = max(yRU)
    print("maxRD: %s" %maxRU)
    
    p_EmaxRU = x[np.argmax(yRU)]
    print("p_E for RP max util: %s" %p_EmaxRU)
    print("")
    
#     test = dRU.subs({rn:n, rPR:PR, rp_a:p_a,rpi:pi,rp_v:p_v,rC_f:C_f,rec:ec})
    p_Einflection = p_EInflection(n,PR,p_a,pi,p_v,C_f,ec)
    
    
    print("p_E:\t"+str(p_Einflection))
#     print("inflection test: %s" %test)

    plt.legend()
    plt.show

In [ ]:
def p_EInflection(n,PR,p_a,pi,p_v,C_f,ec):
    
    D =   pi * PR + (pi* n)
    p2=1-p_a
    
    
    p_EInflectionTop = -pi*p_v*p_a
    p_EInflectionTop += pi*(p_v-1)
    p_EInflectionTop += pi*p_v*p2*(p_a**n - 1)
    p_EInflectionTop += p_v*p2*(p_a**n) *D
    p_EInflectionTop -= p_v*(p_a**n -1) *(pi-C_f)
    p_EInflectionTop -= p_v*(p_a**n) *(C_f+D)
    p_EInflectionTop -= (p_v-1)*(pi-C_f)
    
    p_EInflectionBottom = -2*(p_v*p_a*ec)
    
    
  
    p_EInflectionBottom += (p_v-1)*2*ec
    p_EInflectionBottom += p_v*p2*(p_a**n - 1)*2*ec
    p_EInflectionBottom += p_v*p2*(p_a**n) *2*ec
 
    p_EInflectionFinal=p_EInflectionTop/p_EInflectionBottom

  
    return p_EInflectionFinal

In [ ]:
#pi,B, computation, fine, cj, cd, m):
# wn=widgets.IntSlider(min=0,max=10,step=1,value=4, description='n', continuous_update=False)
# wPR=widgets.FloatSlider(min=0,max=100,step=0.01,value=4, continuous_update=False)
# wNDPR=widgets.FloatSlider(min=0,max=100,step=0.01,value=4, continuous_update=False)

# wpi =widgets.IntSlider(min=0,max=100,step=1,value=4, continuous_update=False)
# wB =widgets.IntSlider(min=0,max=100,step=1,value=5, continuous_update=False)
# wp_v = widgets.FloatSlider(min=0,max=1,step=0.01,value=0.5, continuous_update=False)
# wC_v =widgets.IntSlider(min=0,max=100,step=1,value=2, continuous_update=False)
wroi =widgets.FloatSlider(min=0,max=1,step=0.01,value=0.5, continuous_update=False)
wInsurance =widgets.FloatSlider(min=0,max=1,step=0.01,value=0, continuous_update=False)
wC_f =widgets.FloatSlider(min=0,max=100,step=1,value=0, continuous_update=False)
wp_a =widgets.FloatSlider(min=0,max=1,step=0.01,value=0.8, continuous_update=False)



#wp_E = widgets.FloatSlider(min=0,max=1,step=.1,value=1,description='P(run)', continuous_update=False) # probability that RP runs correctly



interactive_plotRP = widgets.interactive(plotRP,n=wn,PR=wPR, NDPR=wNDPR, p_a=wp_a, pi=wpi, b=wB, p_v=wp_v, C_v=wC_v,roi=wroi, Insurance=wInsurance, C_f=wC_f)
output = interactive_plotRP.children[-1]
# output.layout.height = '600px'
interactive_plotRP

In [ ]:
def RPp_EFinder(n,PR,p_a,pi,p_v,C_f,ec):
    p2=1-p_a
    if p_v==0: #
        if (C_f  >ec ):
            return 1
        elif (C_f  < ec ):
            return 0
        
    fraudCost=C_f
    execCost=ec
    D=pi * PR + (pi * n)
        
    #val finds if p_E0 or p_E1 is higher
    val = p_v*p_a*(pi-execCost)
    val +=p_v*p2*(1-p_a**n)*(pi-execCost)
    val +=p_v*p2*(p_a**n)*(-D-execCost)
    val +=(1-p_v) * (fraudCost - execCost)
    val -=p_v*(1-p_a**n)*(pi-fraudCost)
    val -= p_v*(p_a**n)*(-D-fraudCost)
    
    if val>=0:
        p_E=1
    elif val<0:
        p_E=0
   
   
    
    return p_E
    
   

In [ ]:
test = RPp_EFinder(1,1,1,2,3,4,5)

In [ ]:
def RPNDoptimal(n,PR,NDPR,p_a,pi,p_v,C_f,ec,I):
    p_Enew = RPp_EFinder(n,PR,p_a,pi,p_v,C_f,ec)
    
    # Temporary fix
#     ec = pi*(1-roi)
#     D = pi * PR + (pi * n)
#     I = D*I
    
    RPutil = RPU(n,PR,NDPR,p_a,pi,p_v,C_f,p_Enew, ec, I)
    return(RPutil)

In [ ]:
def plotnonDetFINAL(n,PR,NDPR,C_r, pi,b, p_ETarget,p_v=1,C_v=0, p_E=1,roi=1,Insurance=0,C_f=0):   
    plt.figure(2,figsize=(10,5))
    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    x,step = np.linspace(0,1,101,retstep=True)
    print("step size: %s" %step)
    yJND = list(map(lambda p_a: JCU(n,PR,NDPR,C_r,  p_a,b,pi,p_v=p_v,C_v=C_v, p_E=p_E),x))
    #yRND = list(map(lambda p_a: RPNDU(n,PR,NDPR, p_a,pi,p_v=p_v, p_E=p_E,roi=roi,I=Insurance),x))
    
    ec = pi*(1-roi)
    D = pi * PR + (pi * n)
    I = D*Insurance
    
    yRNDD = list(map(lambda p_a: RPU(n,PR,NDPR,p_a,pi,p_v=p_v,C_f=C_f,p_E=p_E,ec=ec,I=I),x))
    yRNDD0 = list(map(lambda p_a: RPU(n,PR,NDPR,p_a,pi,p_v=p_v,C_f=C_f,p_E=0,ec=ec,I=I),x))
    yRNDD1 = list(map(lambda p_a: RPU(n,PR,NDPR,p_a,pi,p_v=p_v,C_f=C_f,p_E=1,ec=ec,I=I),x))
    
    yRNDoptimal =list(map(lambda p_a: RPNDoptimal(n,PR,NDPR,p_a,pi,p_v=p_v,C_f=C_f,ec=ec,I=I),x))
    
    #                        RPp_EFinder(n, PR, p_a, pi, p_v,   C_f,     roi):
    yp_E = list(map(lambda p_a: RPp_EFinder(n, PR, p_a, pi, p_v=p_v, C_f=C_f, ec=ec), x))
    
    #plt.plot(x,yJND, label="JUND")
    #plt.plot(x,yRND, label="RUND")
    plt.plot(x,yRNDD, label="RUNDD")
    
    plt.plot(x,yp_E, label="p_E Cheat Amount")
    plt.plot(x,yRNDD0, label="RUNDD, p_E=0 CHEATING")
    plt.plot(x,yRNDD1, label="RUNDD, p_E=1 FAIR")
    plt.plot(x,yRNDoptimal, label="RUNDoptimal")
    
    plt.xlabel('P(a1)')
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show
    
    print("n: %s" %n)
    print("Price: %s" %pi)
    D = pi * PR + pi*n
    print("D: %s" %D)
    damages = pi + C_r
    
    maxJND = max(yJND)
    p_aMax = x[yJND.index(maxJND)]
    p2 = 1 - p_aMax
    print("P(a1) for JCmax/RPmin util: %s" %p_aMax)
    print("")
    
    print("Resource Provider")
    execCost = pi*(1-roi)
    requstedPrice = execCost + D*Insurance   
    print("requstedPrice: %s" %requstedPrice)   
    print("pi > requstedPrice, Matchable? : %s" %(pi>requstedPrice))
    
    print("")
    p_apayout = p_v*p_aMax*pi
    P2payout = p_v*p2*(1-p_aMax**n)*damages
    TPayout = p_apayout+P2payout
    RPcost = pi*(1-roi) 
    NDI = D*Insurance
    NDloss = p_v*(p2)*(p_aMax**n)*D
    Tcost = RPcost + NDloss - NDI
    I = p_v*(p2)*(p_aMax**n)
    
    p_a=p_ETarget
    p2=1-p_a #1-p_a
    
    fraudCost=C_f
    
    p_Epoint =  - (fraudCost - execCost)/(p_a*(pi-execCost)+p2*(1-p_a**n)*(pi-execCost)+p2*(p_a**n)*(-D-execCost)-(1-p_a**n)*(pi-fraudCost)- (p_a**n)*(-D-fraudCost) )

    
    optimalp_E = RPp_EFinder(n,PR,p_a,pi,p_v,C_f,roi)

    print("p_E INFLECTION POINT: (p_a = "+str(p_ETarget)+"):\t necessary P = "+str(p_Epoint))
    
   
    print("Optimal p_E: (p_a = "+str(p_ETarget)+"):\t  p_Eoptimal = "+str(optimalp_E))
    
    
    
    print("max ND JC utility: %s" %maxJND)    
    
    
    with open('RP_file.csv', mode='w') as RPfile:
        RPwriter = csv.writer(RPfile, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        RPwriter.writerow(['p_a','RND0', 'RND1', 'RNDoptimal', 'p_Eoptimal'])
        for i in range(101):
            RPwriter.writerow([i/100-1/100, yRNDD0[i], yRNDD1[i], yRNDoptimal[i], yp_E[i] ])

   

In [ ]:
# ui = widgets.HBox([wn,wPR,wNDPR,wpi,wp_v,wcj,wp_E,wroi,winsurance])
#wp_ETarget = widgets.FloatSlider(min=0,max=1,step=.01,value=1, description='p_E Inflection Target',continuous_update=False)#Non-deterministic Penalty rate

ui1 = widgets.HBox([wn,wPR,wNDPR,wC_r])
ui2 = widgets.HBox([wB,wpi,wp_v,wC_v])
ui3 = widgets.HBox([wC_f,wp_E,wroi,winsurance])
ui4 = widgets.HBox([wp_ETarget])

# interactive_plot = widgets.interactive(plot, n=wn,PR=wPR,NDPR=wNDPR,pi=wpi,p_v=wp_v,cj=wcj, p_E=wp_E,roi=wroi,Insurance=winsurance);
interactive_plotFinal = widgets.interactive_output(plotnonDetFINAL,{'n':wn,'PR':wPR,'NDPR':wNDPR,'C_r':wC_r, 'pi':wpi,'b':wB, 'p_ETarget':wp_ETarget, 'p_v':wp_v,'C_v':wC_v, 'p_E':wp_E,'roi':wroi,'Insurance':winsurance,'C_f':wC_f});

interactive_plotFinal.layout.height = '600px'

display(interactive_plotFinal,ui1,ui2,ui3,ui4)
# interactive_plotFinal


# Derivative of JPNDU wrt p_a
   Simplify: Following Equation must not be true for 0<=p_a<1, and p_a
   
   p_v*(B-pi-C_v)
   + p_v*p_E*(1-p_a)
   + n *p_v*(p_E-1)*p_a**(n - 1) *( (-(NDD + C_r + C_v)) - (C_v-r) ) 
   + (B-NDD-C_r-C_v)*p_v*p_E * (n * (1-p_a)*p_a**(n-1)  -  (1-p_a**n)  )
   =0


In [ ]:
def OptimalP(fraudcost,execCost, p_a, pi, n, D):
    
    p2=1-p_a
    fraudcost
    p_a=n/(n+1)

    top = - (fraudcost - execCost)
    bot = p_a*(pi-execCost) +p2*(1-p_a**n)*(pi-execCost) +p2*(p_a**n)*(-D-execCost) -(1-p_a**n)*(pi-fraudcost) - (p_a**n)*(-D-fraudcost)

    rightside=top/bot

    p_v=rightside/(1-rightside)
    
    p_v=p_v+0.001
    
    if (p_v<0):
        p_v=0
    elif (p_v>1):
        p_v=1

    return p_v
    
    

In [ ]:
def pp_Efinder(n,PR,p_a,pi,C_f,ec, D):
    execCost=ec
    fraudcost=C_f
    p_v=OptimalP(fraudcost,execCost, p_a, pi, n, D)
  
    
    p_E=RPp_EFinder(n,PR,p_a,pi,p_v,C_f,ec)

    
    return p_E

In [ ]:
def pRPUtility(n, PR, p_a, pi, fraudcost, execCost, D, roi, I):    
    p_v=OptimalP(fraudcost,execCost, p_a, pi, n, D)

    p_E=pp_Efinder(n,PR,p_a,pi,fraudcost, execCost, D)

    RPutility = RPU(n, PR, PR, p_a, pi, p_v,fraudcost, p_E, execCost, I)
              # RPU(n,PR,NDPR,p_a,pi,p_v=1,C_f=0,p_E=1, ec=0, I=0):
    RPutility0 = RPU(n, PR, PR, p_a, pi, p_v,fraudcost, 0, execCost, I)    
    RPutility1 = RPU(n, PR, PR, p_a, pi, p_v,fraudcost, 1, execCost, I)    
          
    #RPutility=-999
    return [RPutility,RPutility0, RPutility1]

In [ ]:
def plotp(fraudcost,execCost, p_a, pi, n, PR, D,roi, I):  
    plt.figure(2,figsize=(10,5))
    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    x,step = np.linspace(0,1,51,retstep=True)
    print("step size: %s" %step)
    
    yP =   list(map(lambda execCost: OptimalP(fraudcost,execCost, p_a, pi, n, D),x))
    yPp_E =  list(map(lambda execCost: pp_Efinder(n,PR,p_a,pi,fraudcost,execCost, D),x))
    #yPp_E1 = list(map(lambda execCost: RPp_EFinder(n,PR,p_a,pi,1,fraudcost,execCost),x))
    #yPp_E0 = list(map(lambda execCost: RPp_EFinder(n,PR,p_a,pi,0,fraudcost,execCost),x))
    yRPU = list(map(lambda execCost: pRPUtility(n,PR,p_a,pi,fraudcost,execCost, D,roi,I),x))
                                    #pRPUtility(n, PR, p_a, pi, fraudcost, execCost, D, roi, I):       
    #yJND = list(map(lambda p_a: JCU(n,PR,NDPR, p_a,b,pi,p_v=p_v,C_v=C_v, p_E=p_E),x))
#     yRND = list(map(lambda p_a: RPNDU(n,PR,NDPR, p_a,pi,p_v=p_v, p_E=p_E,roi=roi,I=Insurance),x))
    
    #ec = pi*(1-roi)
    #D = pi * PR + (pi * n)
    #I = D*Insurance

    #yRNDD = list(map(lambda p_a: RPU(n,PR,NDPR,p_a,pi,p_v=p_v,C_f=C_f,p_E=p_E,ec=ec,I=I),x))
    
    #plt.plot(x,yPp_E0, label="Optimal p_E when p_v=0")
    #plt.plot(x,yPp_E1, label="Optimal p_E when p_v=1")
    plt.plot(x,yP, label="Optimal P")
    plt.plot(x,yPp_E, label="p_E based on optimal P")
    
    Rp_a = np.array(yRPU)[:,2]
   
    RP0 = np.array(yRPU)[:,1]
    RPopt = np.array(yRPU)[:,0] 
    
    plt.plot(x,RPopt, label="RP Utility with Optimal P, Optimal p_E")
    plt.plot(x,RP0, label="RP Utility with Optimal P, p_E=0")
    plt.plot(x,Rp_a, label="RP Utility with Optimal P, p_E=1")
    #plt.plot(x,yJND, label="JUND")
    #plt.plot(x,yRND, label="RUND")
    #plt.plot(x,yRNDD, label="RUNDD")
    plt.xlabel('execution Cost')
    plt.ylabel('P (verification rate)') 
    
    plt.legend()
    plt.show   
   

In [ ]:
# ui = widgets.HBox([wn,wPR,wNDPR,wpi,wp_v,wcj,wp_E,wroi,winsurance])

wFraudcost = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='fraudcost', continuous_update=False) #Penalty Rate
wexecCost = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='execCost', continuous_update=False) #Penalty Rate
# wpi = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='pi', continuous_update=False) #Penalty Rate

wD = widgets.FloatSlider(min=0,max=300,step=.5,value=1, description='D', continuous_update=False) #Penalty Rate
wn = widgets.IntSlider(min=0,max=10,step=1,value=1, description='n',continuous_update=False)


ui6 = widgets.HBox([wFraudcost, wpi, wn, wPR])
ui7 = widgets.HBox([wD, wroi, winsurance])

#plotp(fraudcost,execCost, p_a, pi, n, D):  
interactive_plotp = widgets.interactive_output(plotp,{'fraudcost': wFraudcost, 'execCost':wexecCost, 'p_a':wp_a, 'pi':wpi, 'n':wn, 'PR':wPR, 'D':wD, 'roi':wroi, 'I':winsurance});

#interactive_plot = widgets.interactive_output(plot,{'n':wn,'PR':wPR,'NDPR':wNDPR,'pi':wpi,'b':wB,'p_v':wp_v,'C_v':wC_v, 'p_E':wp_E,'roi':wroi,'Insurance':winsurance,'C_f':wC_f });


# output = interactive_plot.children[-1]
# output.layout.height = '900px'
display(interactive_plotp,ui6,ui7)